Data Exploration

In [1]:
import pandas as pd
import json

In [2]:
DATA_FILE = "global_cities_data_set.json"
file_object = open(DATA_FILE,)
data = json.load(file_object)

In [3]:
#Inspect data
print(data['data'][0])
print(data['data'][1])
#print(data['data'][558346])
#print(data['data'][579672])
#print(data['data'][579673])
#print(data['data'][2])


{'year': '2023', 'indicator_name': 'Consumer spending by product / service - Furniture and furnishings, carpets and other floor coverings', 'geography_iso': 'AUT', 'geography_country': 'Austria', 'geographyid': 'AT111', 'geographyname': 'AT111 - Mittelburgenland', 'value_unit': nan, 'databank': 'EUREG', 'value': 28556485.914822906}
{'year': '2023', 'indicator_name': 'Consumer spending by product / service - Household and garden tools and equipment', 'geography_iso': 'AUT', 'geography_country': 'Austria', 'geographyid': 'AT111', 'geographyname': 'AT111 - Mittelburgenland', 'value_unit': nan, 'databank': 'EUREG', 'value': 5781600.787580578}


In [49]:
df = pd.json_normalize(data['data'])
print(df.columns.values)
print(df.dtypes)

df.year = df.year.astype('int32')
print(df.dtypes)

['year' 'indicator_name' 'geography_iso' 'geography_country' 'geographyid'
 'geographyname' 'value_unit' 'databank' 'value']
year                  object
indicator_name        object
geography_iso         object
geography_country     object
geographyid           object
geographyname         object
value_unit            object
databank              object
value                float64
dtype: object
year                   int32
indicator_name        object
geography_iso         object
geography_country     object
geographyid           object
geographyname         object
value_unit            object
databank              object
value                float64
dtype: object


In [37]:

indicator_groups = [
    'Household numbers by income band',
    'Population',
    'Consumer spending by product'
]

indicator_groups_strings = (
    'Household numbers by income band',
    'Population',
    'Consumer spending by product'
)

other_indicators = []

for word in df.indicator_name.unique()[:]:
    if not word.startswith(indicator_groups_strings):
        other_indicators.append(word)

# Create separate files for indicators.
for indicator in other_indicators:
    df_filtered = df[(df['indicator_name'] == indicator)]
    filtered_file_name = "data/" + indicator.replace(" ", "_"). \
       replace(",", "_").replace("/", "_") + '.csv'
    df_filtered.to_csv(filtered_file_name, sep=";", encoding="utf-8")

# Group some indicators into one file.
for indicator_group in indicator_groups:
    df_filtered = df[(df['indicator_name'].str.startswith(indicator_group))]
    filtered_file_name = "data/" + indicator_group + '.csv'
    df_filtered.to_csv(filtered_file_name, sep=";", encoding="utf-8")

df.to_csv("data/total_set.csv", sep=";", encoding="utf-8")

['Total households', 'Average household size', 'Household disposable income, per household, real', 'Total population', 'Household disposable income, real', 'Gross domestic product, real', 'Household disposable income, per household, nominal', 'Homeownership rate', 'Housing permits - multi family', 'Housing permits - single family', 'Housing permits - total', 'Housing starts - multi family', 'Housing starts - single family', 'Housing starts - total', 'Income from employment, nominal', 'Income from rent, dividends and interest, nominal', 'Income taxes, nominal', 'Labor force', 'Labor force participation rate', 'Median household income, real', 'Personal disposable income, per capita, real', 'Personal disposable income, per household, real', 'Personal income, per capita, real', 'Personal income, per household, real', 'Proprietors incomes, nominal', 'Social security payments, nominal', 'Unemployment level', 'Unemployment rate', 'CREA house price index', 'Housing starts', 'Labour force parti

In [ ]:
file_object.close()

In [29]:

df_households = pd.read_csv(
    "data/Household numbers by income band_ext.csv",
    sep=";",
    encoding="utf8")

column_names=[
    'geographyid',
    'year'
]

df_households_ext = pd.DataFrame(columns = column_names)
household_value_colum_names = []

# Create single data points
indicator_names = df_households.indicator_name.unique()
for indicator_name in indicator_names:
    df_select = df_households[df_households.indicator_name == indicator_name] 
    column_name = indicator_name. \
        replace("Consumer spending by product / service - ", ""). \
        replace(",", ""). \
        replace(" ", "_"). \
        replace("-", "")
    print("column_name: ", column_name)
    household_value_colum_names.append(column_name)
    df_select[column_name] = df_select['value']
    frames = [df_households_ext, df_select]
    df_households_ext = pd.concat(frames)

#print(household_value_colum_names)

columns_to_drop = [
    'Unnamed: 0',
    'indicator_name',
    'geography_iso',
    'geography_country',
    'geographyname',
    'value_unit',
    'databank',
    'value'
    ]

df_households_ext = df_households_ext.drop(columns_to_drop, axis = 1)

df_households_ext.to_csv("data/Household numbers by income band_ext2.csv",
    sep=";",
    encoding="utf8")


c:\git\assignment\tmp\python\assignment-venv\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [51]:
import pandas as pd

file_list = [
    'Consumer spending by product',
    'Population',
    'Household numbers by income band'
]

df_data = pd.read_csv(
    "data/Population.csv",
    sep=";",
    encoding="utf8")

print("columns: ", df_data.columns.values)
print("shape: ", df_data.shape)

# df_data = df_data.head(10000)
# print("shape: ", df_data.shape)

consumer_value_colum_names = []

columns_to_drop = [
    'Unnamed: 0',
    'indicator_name',
    'geography_iso',
    'geography_country',
    'geographyname',
    'value_unit',
    'databank',
    'value'
    ]

# Sum values
df_grouped = df_data.groupby(by=['year', 'geographyid']).sum()
# Back to data frame
df_sum = df_grouped.reset_index()

def calculate_ratio(par_year, par_geographyid, par_value):
    df_filtered_sum = df_sum[(df_sum['year'] == par_year) &
        (df_sum['geographyid'] == par_geographyid)].sum()
    #print(df_filter)
    return par_value / df_filtered_sum.values[3]

df_data['population_ratio'] = df_data.apply(
        lambda row : calculate_ratio(
            row['year'],
            row['geographyid'],
            row['value']), axis = 1)

print("shape: ", df_data.shape)


df_data.to_csv("data/Population_ext.csv",
     sep=";",
     encoding="utf8")

print("End")


columns:  ['Unnamed: 0' 'year' 'indicator_name' 'geography_iso' 'geography_country'
 'geographyid' 'geographyname' 'value_unit' 'databank' 'value']
shape:  (270232, 10)
shape:  (270232, 11)


In [9]:
import pandas as pd

file_list = [
    'Consumer spending by product',
    'Population',
    'Household numbers by income band'
]

df_consumer_spend = pd.read_csv(
    "data/Consumer spending by product.csv",
    sep=";",
    encoding="utf8")

print("shape: ", df_consumer_spend.shape)

#df_consumer_spend = df_consumer_spend.head(200000)

#print("shape: ", df_consumer_spend.shape)

consumer_value_colum_names = []

columns_to_drop = [
    'Unnamed: 0',
    'indicator_name',
    'geography_iso',
    'geography_country',
    'geographyname',
    'value_unit',
    'databank',
    'value'
    ]

# remove unnecessary columns
#df_consumer_spend = df_consumer_spend.loc[:, ['year', 'geographyid', 'value']]
#df_one = df_consumer_spend[(df_consumer_spend['year'] == 2018) &
#                           (df_consumer_spend['geographyid'] == 'CIV001')]
#print("sum: ", df_one.value.sum())

# Sum values
df_grouped = df_consumer_spend.groupby(by=['year', 'geographyid']).sum()
# Back to data frame
df_sum = df_grouped.reset_index()

#df_sum.to_csv("data/df_sum.csv", sep=";", encoding="utf8")
#sum = df_sum[(df_sum['year'] == 2018) & (df_sum['geographyid'] == 'CIV001')].sum()
#print("sum: ", sum)
#print(df_grouped.apply(lambda g: g[g['geographyid'] == 'CIV001']))

def calculate_ratio(par_year, par_geographyid, par_value):
    df_filtered_sum = df_sum[(df_sum['year'] == par_year) &
        (df_sum['geographyid'] == par_geographyid)].sum()
    #print(df_filter)
    return par_value / df_filtered_sum.values[3]

# def calculate_sum(par_year, par_geographyid, par_value):
#     df_filter = df_sum[(df_sum['year'] == par_year) &
#         (df_sum['geographyid'] == par_geographyid)].sum()
#     print(df_filter)
#     return df_filter.values[3]

df_consumer_spend['spend_ratio'] = df_consumer_spend.apply(
        lambda row : calculate_ratio(
            row['year'],
            row['geographyid'],
            row['value']), axis = 1)

# df_consumer_spend['spend_sum'] = df_consumer_spend.apply(
#         lambda row : calculate_sum(
#             row['year'],
#             row['geographyid'],
#             row['value']), axis = 1)

print("shape: ", df_consumer_spend.shape)

df_consumer_spend.to_csv("data/Consumer spending by product_ext.csv",
     sep=";",
     encoding="utf8")



shape:  (225109, 10)
shape:  (225109, 11)


In [70]:
df_data = pd.read_csv(
    "data/Consumer spending by product_ext.csv",
    sep=";",
    encoding="utf8")

print("shape: ", df_data.shape)

#Todo something weird happening 80000 and 90000
df_data = df_data.head(80000)

print("shape: ", df_data.shape)

column_names = []
df_data_ext = pd.DataFrame()

columns_to_drop = [
    'Unnamed: 0',
    'indicator_name',
    'geography_iso',
    'geography_country',
    'geographyname',
    'value_unit',
    'databank',
    'value',
    'spend_ratio'
    ]

# Create single data points
indicator_names = df_data.indicator_name.unique()
for indicator_name in indicator_names:
    df_select = df_data[df_data.indicator_name == indicator_name] 
    column_name = indicator_name. \
        replace("Consumer spending by product / service - ", ""). \
        replace(",", ""). \
        replace(" ", "_"). \
        replace("-", ""). \
        lower()
    print("column_name: ", column_name)
    column_names.append(column_name)
    df_select[column_name] = df_select['spend_ratio']
    df_select = df_select.drop(columns_to_drop, axis=1)
    df_select = df_select.loc[:, ("geographyid", "year", column_name)]

    if (len(df_data_ext) == 0):
        df_data_ext = df_select
    else:
        df_data_ext = df_data_ext.merge(
            right=df_select,
            on=["geographyid", "year"],
            how="outer")

    print("Shape: ", df_data_ext.shape)

#df_data_ext = df_data_ext.drop(columns_to_drop, axis=1)

df_data_ext.to_csv("data/Consumer spending by product_ext2.csv",
    sep=";",
    encoding="utf8")

print("End")

c:\git\assignment\tmp\python\assignment-venv\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\git\assignment\tmp\python\assignment-venv\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


shape:  (225109, 12)
shape:  (80000, 12)
column_name:  furniture_and_furnishings_carpets_and_other_floor_coverings
Shape:  (11429, 3)
column_name:  household_and_garden_tools_and_equipment
Shape:  (11429, 4)
column_name:  household_appliances
Shape:  (11429, 5)
column_name:  household_furnishings_household_equipment_and_other_housing_expenditure__total
Shape:  (11429, 6)
column_name:  household_glassware_tableware_and_household_utensils
Shape:  (11429, 7)
column_name:  household_textiles
Shape:  (11429, 8)
column_name:  routine_household_maintenance_goods_and_services
Shape:  (11429, 9)
End


In [ ]:


df_merged1 = df_households_ext.merge(
    df_population_ext,
    on=["geographyid", "year"])

df_consumer_spend_ext
df_population_ext
df_households_ext

df_merged1.to_csv("data/df_merged.csv", sep=";", encoding="utf8")